## Setup

In [1]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import numpy as np
import pandas as pd
import random
import tensorflow as tf
import isuelogit as isl
import glob

In [2]:
# Path management
main_dir = str(Path(os.path.abspath('')).parents[1])
os.chdir(main_dir)
print('main dir:', main_dir)

isl.config.dirs['read_network_data'] = "input/network-data/fresno/"

main dir: /Users/pablo/github/gisuelogit


In [3]:
# Internal modules
from src.gisuelogit.models import UtilityParameters, BPRParameters, ODParameters, GISUELOGIT, compute_rr
from src.gisuelogit.visualizations import plot_predictive_performance, plot_convergence_estimates, plot_top_od_flows_periods, plot_utility_parameters_periods, plot_rr_by_period, plot_rr_by_period_models, plot_total_trips_models
from src.gisuelogit.networks import load_k_shortest_paths, read_paths, build_fresno_network, \
    Equilibrator, sparsify_OD, ColumnGenerator, read_OD
from src.gisuelogit.etl import get_design_tensor, get_y_tensor, data_curation, temporal_split, add_period_id, get_tensors_by_year
from src.gisuelogit.descriptive_statistics import mse, btcg_mse, mnrmse, nrmse

In [4]:
# Seed for reproducibility
_SEED = 2023
np.random.seed(_SEED)
random.seed(_SEED)
tf.random.set_seed(_SEED)

## Build Fresno network

In [5]:
fresno_network = build_fresno_network()

## Read OD matrix

In [6]:
read_OD(network=fresno_network, sparse=True)

Matrix Q (1789, 1789) read in 0.0[s] with sparse format
66266.3 trips were loaded among 6970 o-d pairs



## Read paths

In [7]:
#read_paths(network=fresno_network, update_incidence_matrices=True, filename='paths-fresno.csv')
read_paths(network=fresno_network, update_incidence_matrices=True, filename = 'paths-full-model-fresno.csv')

# For quick testing
# Q = fresno_network.load_OD(sparsify_OD(fresno_network.Q, prop_od_pairs=0.99))
# load_k_shortest_paths(network=fresno_network, k=2, update_incidence_matrices=True)

Reading paths: |################----| 81.5% 

KeyboardInterrupt: 

## Read spatiotemporal data

In [ ]:
folderpath = isl.config.dirs['read_network_data'] + 'links/spatiotemporal-data/'
df = pd.concat([pd.read_csv(file) for file in glob.glob(folderpath + "*link-data*")], axis=0)

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
# Select data from Tuesday to Thursday
df = df[df['date'].dt.dayofweek.between(1, 3)]
# df = df[df['date'].dt.year == 2019]

In [ ]:
# Add period id for timevarying estimation

period_feature = 'hour'

df['period'] = df['date'].astype(str) + '-' + df[period_feature].astype(str)
# df['period'] = df.period.map(hash)

df = add_period_id(df, period_feature='hour')

period_keys = df[[period_feature,'period_id']].drop_duplicates().reset_index().drop('index',axis =1).sort_values('hour')
print(period_keys)

In [ ]:
df.head()

In [ ]:
# Units in miles per hour
df[['speed_ref_avg','speed_hist_avg','speed_max']].describe()

In [ ]:
df.columns

## Data curation

In [ ]:
df['tt_ff'] = np.where(df['link_type'] != 'LWRLK', 0,df['length']/df['speed_ref_avg'])
df.loc[(df.link_type == "LWRLK") & (df.speed_ref_avg == 0),'tt_ff'] = float('nan')

df['tt_avg'] = np.where(df['link_type'] != 'LWRLK', 0,df['length']/df['speed_hist_avg'])
df.loc[(df.link_type == "LWRLK") & (df.speed_hist_avg == 0),'tt_avg'] = float('nan')

tt_sd_adj = df.groupby(['period_id','link_key'])[['tt_avg']].std().reset_index().rename(columns = {'tt_avg': 'tt_sd_adj'})

df = df.merge(tt_sd_adj, on = ['period_id','link_key'])

df = data_curation(df)

df['tt_sd'] = df['tt_sd_adj']

In [ ]:
# Units of travel time features are converted from hours to minutes
df['tt_sd'] = df['tt_sd']*60
df['tt_avg'] = df['tt_avg']*60
df['tt_ff'] = df['tt_ff']*60

In [ ]:
df[['speed_ref_avg','speed_hist_avg', 'tt_ff', 'tt_avg','tt_sd_adj']].describe()

## Utility function

In [ ]:
features_Z = ['tt_sd', 'median_inc', 'incidents', 'bus_stops', 'intersections']
# features_Z = ['speed_sd', 'median_inc', 'incidents', 'bus_stops', 'intersections']


utility_parameters = UtilityParameters(features_Y=['tt'],
                                       features_Z=features_Z,
                                       initial_values={'tt': 0, 'c': 0, 's': 0, 'psc_factor': 0,
                                                       'fixed_effect': np.zeros_like(fresno_network.links)},
                                       signs={'tt': '-', 'tt_sd': '-', 'median_inc': '+', 'incidents': '-',
                                              'bus_stops': '-', 'intersections': '-'},
                                       trainables={'psc_factor': False, 'fixed_effect': False},
                                       )

utility_parameters.constant_initializer(0)

## Data processing

In [ ]:
n_links = len(fresno_network.links)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['year'] = df.date.dt.year
X, Y = {}, {}

In [ ]:
df.query('year == 2019')[['counts', 'tt_ff', 'tt_avg', 'tf_inrix', 'tt_sd']].describe()

In [ ]:
df.query('year == 2020')[['counts', 'tt_ff', 'tt_avg', 'tf_inrix', 'tt_sd']].describe()

In [ ]:
# Set free flow travel times
tt_ff_links = df.groupby('link_key')['tt_ff'].min()
for link in fresno_network.links:
    fresno_network.links_dict[link.key].performance_function.tf = float(tt_ff_links[tt_ff_links.index==str(link.key)])

In [ ]:
# This correlation should be positive
df[['counts','tt_avg']].corr()

## EDA

In [ ]:
# To check that there is a balanced amount of observations per date
obs_date = df.groupby('date')['hour'].count()

In [ ]:
# Stats by date
df.groupby('date')[['speed_sd','speed_avg', 'counts']].mean().assign(total_obs = obs_date)

### Link attributes

In [ ]:
df[features_Z].describe()

## Training and validation sets

In [ ]:
# Include only data between 4pm and 5pm
# X, Y = get_tensors_by_year(df, features_Z = features_Z, network = fresno_network)
#X, Y = get_tensors_by_year(df[df.hour.isin([7])], features_Z = features_Z, network = fresno_network)
X, Y = get_tensors_by_year(df[df.hour.isin([16])], features_Z = features_Z, network = fresno_network)
# X, Y = get_tensors_by_year(df[df.hour.isin([15,16,17])], features_Z = features_Z, network = fresno_network)
#X, Y = get_tensors_by_year(df[df.hour.isin([6,7,8])], features_Z = features_Z, network = fresno_network)

# Include hourly data between 6AM and 8PM (15 hour intervals)
# XT, YT = get_tensors_by_year(df, features_Z = features_Z)
#XT, YT = get_tensors_by_year(df[df.hour.isin(range(15,16))], features_Z = features_Z, network = fresno_network)
XT, YT = get_tensors_by_year(df[df.hour.isin([6,7,8, 15,16,17])], features_Z = features_Z, network = fresno_network)
#XT, YT = get_tensors_by_year(df[df.hour.isin([6,7,8])], features_Z = features_Z, network = fresno_network)

# X_train, X_test, Y_train, Y_test = temporal_split(X[2019].numpy(), Y[2019].numpy(), n_days = X[2019].shape[0])

# X_train, X_test, Y_train, Y_test = X[2020], X[2019], Y[2020], Y[2019]
X_train, X_test, Y_train, Y_test = X[2019], X[2020], Y[2019], Y[2020]
XT_train, XT_test, YT_train, YT_test = XT[2019], XT[2020], YT[2019], YT[2020]

# Remove validation set to reduce computation costs
X_test, Y_test = None, None
XT_test, YT_test = None, None

## Network equilibrium predictor

In [ ]:
equilibrator = Equilibrator(
    network=fresno_network,
    # paths_generator=paths_generator,
    utility=utility_parameters,
    max_iters=100,
    method='fw',
    iters_fw=50,
    accuracy=1e-4,
)

column_generator = ColumnGenerator(equilibrator=equilibrator,
                                   utility=utility_parameters,
                                   n_paths=0,
                                   ods_coverage=0.1,
                                   ods_sampling='sequential',
                                   # ods_sampling='demand',
                                   )

## Configuration

In [ ]:
# For testing
# _EPOCHS = {'learning': 4, 'equilibrium': 2}
# _EPOCHS = {'learning': 10, 'equilibrium': 5}
_EPOCHS = {'learning': 150, 'equilibrium': 100}
_BATCH_SIZE = 16
_LR = 5e-1
_RELATIVE_GAP = 1e-6
_XTICKS_SPACING = 50
_EPOCHS_PRINT_INTERVAL = 1

# When adding fixed effects, all parameters of the utility function are idenfiable but provided that the utility coefficients are period specific. The
# only attribute that is always identifiable is the standard deviation of travel time because it changes over hours for same link. Overall, removing fixed effect ease the identification of the parameters of the utility function.
_FIXED_EFFECT = False
# features_Z = ['tt_sd']

# Excluding historic OD gives more freedom for the model to find an equilibria and minimize reconstruction error
# _LOSS_WEIGHTS ={'od': 0, 'tt': 1, 'flow': 1, 'eq_flow': 1, 'ntrips': 1, 'prop_od': 1}
_LOSS_WEIGHTS ={'od': 0, 'tt': 1, 'flow': 1, 'eq_flow': 1, 'ntrips': 0, 'prop_od': 0}

# _LOSS_METRIC  = mnrmse
# _MOMENTUM_EQUILIBRIUM = {'lue': 0.95, 'odlue': 0.95, 'odlulpe': 0.95, 'tvodlulpe':0.95}

# NRMSE encourages a larger reduction in link flow loss and it does not requires to add much momentum to the equilibrium loss component
_LOSS_METRIC  = nrmse
_MOMENTUM_EQUILIBRIUM = {'lue': 0.99, 'odlue': 0.99, 'odlulpe': 0.99, 'tvodlulpe':0.99}
# _MOMENTUM_EQUILIBRIUM = {'lue': 0.95, 'odlue': 0.95, 'odlulpe': 0.95, 'tvodlulpe':0.95}
#_MOMENTUM_EQUILIBRIUM = {'lue': 0.8, 'odlue': 0.8, 'odlulpe': 0.8, 'tvodlulpe':0.8}

# Including historic OD matrix
# _LOSS_WEIGHTS ={'od': 1, 'tt': 1, 'flow': 1, 'eq_flow': 1}
# _MOMENTUM_EQUILIBRIUM = 0.99

# _LOSS_METRIC = mse
# _LOSS_WEIGHTS ={'od': 1, 'theta': 0, 'tt': 1e10, 'flow': 1, 'eq_flow': 1}

print(f"_LOSS_WEIGHTS: {_LOSS_WEIGHTS}, _MOMENTUM_EQUILIBRIUM: {_MOMENTUM_EQUILIBRIUM}, "
      f"epochs: { _EPOCHS}, 'LR: {_LR}")

## Models

In [ ]:
run_model = dict.fromkeys(['lue', 'odlue','odlulpe', 'tvodlulpe'], True)
# run_model = dict.fromkeys(['lue', 'odlue','odlulpe', 'tvodlulpe'], False)

# run_model['lue'] = True
# run_model['odlue'] = True
# run_model['odlulpe'] = True
# run_model['tvodlulpe'] = True

In [ ]:
train_results_dfs = {}
test_results_dfs = {}

### Model 1: Estimation of utility function (LUE)

In [ ]:
if run_model['lue']:
    print('\nLUE: Estimation of utility function')

    # _MOMENTUM_EQUILIBRIUM = 1

    optimizer = tf.keras.optimizers.Adam(learning_rate=_LR)

    _FIXED_EFFECT = False

    utility_parameters = UtilityParameters(features_Y=['tt'],
                                           features_Z=features_Z,
                                           # initial_values ={'tt': -1, 'tt_sd': -1,
                                           #                  'median_inc': 1,
                                           #                  'incidents': -1, 'bus_stops': -1, 'intersections': -1,
                                           #                  'psc_factor': 0, 'fixed_effect': np.zeros_like(fresno_network.links)},
                                           initial_values={'psc_factor': 0,
                                                           'fixed_effect': np.zeros_like(fresno_network.links)},
                                           signs={'tt': '-', 'tt_sd': '-', 'median_inc': '+', 'incidents': '-',
                                                  'bus_stops': '-', 'intersections': '-'},
                                           trainables={'psc_factor': False, 'fixed_effect': _FIXED_EFFECT,
                                                       'tt': True, 'tt_sd': True,
                                                       # 'median_inc': False, 'incidents': False,
                                                       # 'bus_stops': False, 'intersections': False,
                                                       'median_inc': True, 'incidents': True,
                                                       'bus_stops': True, 'intersections': True
                                                       },
                                           time_varying = True
                                           )

    bpr_parameters = BPRParameters(keys=['alpha', 'beta'],
                                   initial_values={'alpha': 0.15, 'beta': 4},
                                   trainables=dict.fromkeys(['alpha', 'beta'], False),
                                   )

    od_parameters = ODParameters(key='od',
                                 initial_values=fresno_network.q.flatten(),
                                 true_values=fresno_network.q.flatten(),
                                 historic_values={10: fresno_network.q.flatten()},
                                 trainable=False)

    equilibrator = Equilibrator(
        network=fresno_network,
        utility=utility_parameters,
        max_iters=100,
        method='fw',
        iters_fw=50,
        accuracy=1e-4,
    )

    column_generator = ColumnGenerator(equilibrator=equilibrator,
                                       utility=utility_parameters,
                                       n_paths=0,
                                       ods_coverage=0.1,
                                       ods_sampling='sequential',
                                       )

    lue = GISUELOGIT(
        key='lue',
        network=fresno_network,
        dtype=tf.float64,
        equilibrator = equilibrator,
        utility=utility_parameters,
        bpr=bpr_parameters,
        od=od_parameters,
        n_periods = len(np.unique(X_train[:,:,-1].numpy().flatten()))
    )

    train_results_dfs['lue'], test_results_dfs['lue'] = lue.train(
        X_train, Y_train, X_test, Y_test,
        optimizer=optimizer,
        batch_size=_BATCH_SIZE,
        threshold_relative_gap=_RELATIVE_GAP,
        momentum_equilibrium = _MOMENTUM_EQUILIBRIUM['lue'],
        loss_weights= dict(_LOSS_WEIGHTS, od = 0),
        loss_metric=_LOSS_METRIC,
        epochs_print_interval=_EPOCHS_PRINT_INTERVAL,
        epochs=_EPOCHS)

    # Relative loss curves over epochs
    plot_predictive_performance(train_losses=train_results_dfs['lue'], val_losses=test_results_dfs['lue'],
                                xticks_spacing = _XTICKS_SPACING)

    # Compute utility parameters over time (heatmap) and value of travel time reliability (lineplot)
    theta_df = plot_utility_parameters_periods(lue, period_keys = period_keys, period_feature='hour')

    plot_rr_by_period(lue, period_keys, period_feature='hour')

    # Average reliability ratio over epochs
    plot_convergence_estimates(estimates=train_results_dfs['lue'].\
                       assign(rr = train_results_dfs['lue']['tt_sd']/train_results_dfs['lue']['tt'])[['epoch','rr']],
                           xticks_spacing = _XTICKS_SPACING)

    plt.ylabel('average reliability ratio')

    # Distribution of fixed effects
    sns.displot(pd.DataFrame({'fixed_effect':np.array(lue.fixed_effect)}),
        x="fixed_effect", multiple="stack", kind="kde", alpha = 0.8)

    #print(f"theta = {dict(zip(utility_parameters.true_values.keys(), list(lue.theta.numpy())))}")
    print(f"theta:\n\n\ {theta_df.assign(rr = theta_df['tt_sd']/theta_df['tt'])}")
    print(f"alpha = {lue.alpha: 0.2f}, beta  = {lue.beta: 0.2f}")
    print(f"Avg abs diff of observed and estimated OD: {np.mean(np.abs(lue.q - fresno_network.q.flatten())): 0.2f}")
    print(f"Avg observed OD: {np.mean(np.abs(fresno_network.q.flatten())): 0.2f}")

### Model 2: OD + utility estimation with historic OD (ODLUE)

In [ ]:
if run_model['odlue']:

    # _MOMENTUM_EQUILIBRIUM = 0.99

    print('\nODLUE: OD + utility estimation with historic OD')

    # _RELATIVE_GAP = 1e-4\
    # _XTICKS_SPACING = 50

    optimizer = tf.keras.optimizers.Adam(learning_rate=_LR)
    # optimizer = tf.keras.optimizers.Adagrad(learning_rate=_LR)
    _FIXED_EFFECT = False

    utility_parameters = UtilityParameters(features_Y=['tt'],
                                           features_Z=features_Z,
                                           # initial_values ={'tt': -1, 'tt_sd': -1,
                                           #                  'median_inc': 1,
                                           #                  'incidents': -1, 'bus_stops': -1, 'intersections': -1,
                                           #                  'psc_factor': 0, 'fixed_effect': np.zeros_like(fresno_network.links)},
                                           initial_values={'psc_factor': 0,
                                                           'fixed_effect': np.zeros_like(fresno_network.links)},
                                           signs={'tt': '-', 'tt_sd': '-', 'median_inc': '+', 'incidents': '-',
                                                  'bus_stops': '-', 'intersections': '-'},
                                           trainables={'psc_factor': False, 'fixed_effect': _FIXED_EFFECT,
                                                       'tt': True, 'tt_sd': True,
                                                       # 'median_inc': False, 'incidents': False,
                                                       # 'bus_stops': False, 'intersections': False,
                                                       'median_inc': True, 'incidents': True,
                                                       'bus_stops': True, 'intersections': True
                                                       },
                                           time_varying = True
                                           )

    bpr_parameters = BPRParameters(keys=['alpha', 'beta'],
                                   initial_values={'alpha': 0.15, 'beta': 4},
                                   trainables=dict.fromkeys(['alpha', 'beta'], False),
                                   )

    od_parameters = ODParameters(key='od',
                                 initial_values=fresno_network.q.flatten(),
                                 historic_values={10: fresno_network.q.flatten()},
                                 # total_trips={10: 1e5},
                                 trainable=True)

    equilibrator = Equilibrator(
        network=fresno_network,
        # paths_generator=paths_generator,
        utility=utility_parameters,
        max_iters=100,
        method='fw',
        iters_fw=50,
        accuracy=1e-4,
    )

    column_generator = ColumnGenerator(equilibrator=equilibrator,
                                       utility=utility_parameters,
                                       n_paths=0,
                                       ods_coverage=0.1,
                                       ods_sampling='sequential',
                                       # ods_sampling='demand',
                                       )

    odlue = GISUELOGIT(
        key='odlue',
        network=fresno_network,
        dtype=tf.float64,
        equilibrator=equilibrator,
        column_generator=column_generator,
        utility=utility_parameters,
        bpr=bpr_parameters,
        od=od_parameters,
        n_periods = len(np.unique(X_train[:,:,-1].numpy().flatten()))
    )

    train_results_dfs['odlue'], test_results_dfs['odlue'] = odlue.train(
        X_train, Y_train, X_test, Y_test,
        optimizer=optimizer,
        batch_size=_BATCH_SIZE,
        # generalization_error={'train': False, 'validation': True},
        loss_weights= _LOSS_WEIGHTS,
        loss_metric=_LOSS_METRIC,
        momentum_equilibrium = _MOMENTUM_EQUILIBRIUM['odlue'],
        threshold_relative_gap=_RELATIVE_GAP,
        epochs_print_interval=_EPOCHS_PRINT_INTERVAL,
        epochs=_EPOCHS)

    plot_predictive_performance(train_losses=train_results_dfs['odlue'], val_losses=test_results_dfs['odlue'],
                                xticks_spacing = _XTICKS_SPACING)

    plot_top_od_flows_periods(odlue,
                              historic_od= fresno_network.q.flatten(),
                              period_keys = period_keys,
                              period_feature='hour', top_k=20)

    # Compute utility parameters over time (heatmap) and value of travel time reliability (lineplot)
    theta_df = plot_utility_parameters_periods(odlue, period_keys = period_keys, period_feature='hour')

    plot_rr_by_period(odlue, period_keys, period_feature='hour')

    plot_convergence_estimates(estimates=train_results_dfs['odlue'].\
                       assign(rr = train_results_dfs['odlue']['tt_sd']/train_results_dfs['odlue']['tt'])[['epoch','rr']],
                           xticks_spacing = _XTICKS_SPACING)
    plt.ylabel('average reliability ratio')

    sns.displot(pd.DataFrame({'fixed_effect':np.array(odlue.fixed_effect)}),
            x="fixed_effect", multiple="stack", kind="kde", alpha = 0.8)

    #print(f"theta = {dict(zip(utility_parameters.true_values.keys(), list(odlue.theta.numpy())))}")
    print(f"theta:\n\n\ {theta_df.assign(rr = theta_df['tt_sd']/theta_df['tt'])}")
    print(f"alpha = {odlue.alpha: 0.2f}, beta  = {odlue.beta: 0.2f}")
    print(f"Avg abs diff of observed and estimated OD: {np.mean(np.abs(odlue.q - fresno_network.q.flatten())): 0.2f}")
    print(f"Avg observed OD: {np.mean(np.abs(fresno_network.q.flatten())): 0.2f}")

### Model 3: ODLUE + link specific performance parameters (ODLULPE)

In [ ]:
if run_model['odlulpe']:

    print('\nODLULPE: ODLUE + link performance parameters with historic OD matrix (link specifics alphas and betas)')

    # _MOMENTUM_EQUILIBRIUM = 0.99
    _FIXED_EFFECT = False

    optimizer = tf.keras.optimizers.Adam(learning_rate=_LR)

    # _LR = 5e-1
    # _RELATIVE_GAP = 1e-5

    # Some initializations of the bpr parameters, makes the optimization to fail (e.g. alpha =1, beta = 1). Using a common
    # alpha but different betas for every link make the estimation more stable but there is overfitting after a certain amount of iterations

    bpr_parameters = BPRParameters(keys=['alpha', 'beta'],
                                   initial_values={'alpha': 0.15*np.ones_like(fresno_network.links,dtype = np.float32),
                                                   'beta': 4*np.ones_like(fresno_network.links,dtype = np.float32),
                                                   # 'beta': 4
                                                   },
                                   trainables={'alpha': True, 'beta': True},
                                   # trainables={'alpha': True, 'beta': False},
                                   )

    od_parameters = ODParameters(key='od',
                                 initial_values=fresno_network.q.flatten(),
                                 historic_values={10: fresno_network.q.flatten()},
                                 # total_trips={10: 1e5},
                                 trainable=True
                                 )

    utility_parameters = UtilityParameters(features_Y=['tt'],
                                           features_Z=features_Z,
                                           # initial_values ={'tt': -1, 'tt_sd': -1,
                                           #                  'median_inc': 1,
                                           #                  'incidents': -1, 'bus_stops': -1, 'intersections': -1,
                                           #                  'psc_factor': 0, 'fixed_effect': np.zeros_like(fresno_network.links)},
                                           initial_values={'psc_factor': 0,
                                                           'fixed_effect': np.zeros_like(fresno_network.links)},
                                           signs={'tt': '-', 'tt_sd': '-',
                                                  'median_inc': '+', 'incidents': '-',
                                                  'bus_stops': '-', 'intersections': '-'
                                                  },
                                           trainables={'psc_factor': False, 'fixed_effect': _FIXED_EFFECT,
                                                       'tt': True, 'tt_sd': True,
                                                       # 'median_inc': False, 'incidents': False,
                                                       # 'bus_stops': False, 'intersections': False,
                                                       'median_inc': True, 'incidents': True,
                                                       'bus_stops': True, 'intersections': True
                                                       },
                                           time_varying = True
                                           )

    equilibrator = Equilibrator(
        network=fresno_network,
        # paths_generator=paths_generator,
        utility=utility_parameters,
        max_iters=100,
        method='fw',
        iters_fw=50,
        accuracy=1e-4,
    )

    column_generator = ColumnGenerator(equilibrator=equilibrator,
                                       utility=utility_parameters,
                                       n_paths=0,
                                       ods_coverage=0.1,
                                       ods_sampling='sequential',
                                       # ods_sampling='demand',
                                       )

    odlulpe = GISUELOGIT(
        key='odlulpe',
        network=fresno_network,
        dtype=tf.float64,
        equilibrator=equilibrator,
        column_generator=column_generator,
        utility=utility_parameters,
        bpr=bpr_parameters,
        od=od_parameters,
        n_periods = len(np.unique(X_train[:,:,-1].numpy().flatten()))
    )

    train_results_dfs['odlulpe'], test_results_dfs['odlulpe'] = odlulpe.train(
        X_train, Y_train, X_test, Y_test,
        optimizer=optimizer,
        # generalization_error={'train': False, 'validation': True},
        batch_size=_BATCH_SIZE,
        # loss_weights={'od': 1, 'theta': 0, 'tt': 1, 'flow': 1, 'eq_flow': 1},
        loss_weights=_LOSS_WEIGHTS,
        momentum_equilibrium = _MOMENTUM_EQUILIBRIUM['odlulpe'],
        threshold_relative_gap=_RELATIVE_GAP,
        epochs_print_interval=_EPOCHS_PRINT_INTERVAL,
        loss_metric=_LOSS_METRIC,
        epochs=_EPOCHS)



    plot_predictive_performance(train_losses=train_results_dfs['odlulpe'], val_losses=test_results_dfs['odlulpe'],
                                xticks_spacing = _XTICKS_SPACING)

    plot_convergence_estimates(estimates=train_results_dfs['odlulpe'][['epoch','alpha','beta']],
                                xticks_spacing = _XTICKS_SPACING)

    sns.displot(pd.melt(pd.DataFrame({'alpha':odlulpe.alpha, 'beta': odlulpe.beta}), var_name = 'parameters'),
                x="value", hue="parameters", alpha = 0.8)

    top_q, total_trips_by_hour = plot_top_od_flows_periods(odlulpe,
                                                           historic_od= fresno_network.q.flatten(),
                                                           period_keys = period_keys,
                                                           period_feature='hour', top_k=20)

    # Compute utility parameters over time (heatmap) and value of travel time reliability (lineplot)
    theta_df = plot_utility_parameters_periods(odlulpe, period_keys = period_keys, period_feature='hour')

    plot_rr_by_period(odlulpe, period_keys, period_feature='hour')


    plot_convergence_estimates(estimates=train_results_dfs['odlulpe'].\
                   assign(rr = train_results_dfs['odlulpe']['tt_sd']/train_results_dfs['odlulpe']['tt'])[['epoch','rr']],
                       xticks_spacing = _XTICKS_SPACING)

    plt.ylabel('average reliability ratio')

    sns.displot(pd.DataFrame({'fixed_effect':np.array(odlulpe.fixed_effect)}),
                x="fixed_effect", multiple="stack", kind="kde", alpha = 0.8)

    #print(f"theta = {dict(zip(utility_parameters.true_values.keys(), list(odlulpe.theta.numpy())))}")
    print(f"theta:\n\n\ {theta_df.assign(rr = theta_df['tt_sd']/theta_df['tt'])}")
    print(f"alpha = {np.mean(odlulpe.alpha): 0.2f}, beta  = {np.mean(odlulpe.beta): 0.2f}")
    print(f"Avg abs diff of observed and estimated OD: {np.mean(np.abs(odlulpe.q - fresno_network.q.flatten())): 0.2f}")
    print(f"Avg observed OD: {np.mean(np.abs(fresno_network.q.flatten())): 0.2f}")

### Model 4: ODLULPE with Time Varying OD and Utility Function (TVODLULPE)

In [ ]:
if run_model['tvodlulpe']:
    print('\ntvodlulpe: Time specific utility and OD, link performance parameters')

    _FIXED_EFFECT = True
    _LR = 1e-1

    # Only in this model, we can add fixed effect and have identifiable utility function coefficients

    optimizer = tf.keras.optimizers.Adam(learning_rate=_LR)

    utility_parameters = UtilityParameters(features_Y=['tt'],
                                           features_Z=features_Z,
                                           # initial_values ={'tt': -1, 'tt_sd': -1,
                                           #                  'median_inc': 1,
                                           #                  'incidents': -1, 'bus_stops': -1, 'intersections': -1,
                                           #                  'psc_factor': 0, 'fixed_effect': np.zeros_like(fresno_network.links)},
                                           initial_values={'tt': 0, 'tt_sd': 0, 's': 0, 'psc_factor': 0,
                                                           'fixed_effect': np.zeros_like(fresno_network.links)},
                                           signs={'tt': '-',
                                                  'tt_sd': '-',
                                                  'median_inc': '+', 'incidents': '-',
                                                  'bus_stops': '-', 'intersections': '-'
                                                  },
                                           trainables={'psc_factor': False, 'fixed_effect': _FIXED_EFFECT,
                                                       'tt': True, 'tt_sd': True,
                                                       # 'median_inc': False, 'incidents': False,
                                                       # 'bus_stops': False, 'intersections': False,
                                                       'median_inc': True, 'incidents': True,
                                                       'bus_stops': True, 'intersections': True
                                                       },
                                           time_varying=True,
                                           )

    bpr_parameters = BPRParameters(keys=['alpha', 'beta'],
                                   initial_values={'alpha': 0.15*np.ones_like(fresno_network.links,dtype = np.float32),
                                                   'beta': 4*np.ones_like(fresno_network.links,dtype = np.float32),
                                                   # 'beta': 4
                                                   },
                                   trainables={'alpha': True, 'beta': True},
                                   # trainables={'alpha': True, 'beta': False},
                                   )

    od_parameters = ODParameters(key='od',
                                 initial_values=fresno_network.q.flatten(),
                                 true_values=fresno_network.q.flatten(),
                                 historic_values={10: fresno_network.q.flatten()},
                                 # total_trips={0: 1e5, 1:1e5, 2: 1e5, 9: 1e5, 10: 1e5, 11: 1e5},
                                 # total_trips={i:1e5 for i in [0,1,2,9,10,11]},
                                 time_varying=True,
                                 trainable=True)

    tvodlulpe = GISUELOGIT(
        key='tvodlulpe',
        network=fresno_network,
        dtype=tf.float64,
        utility=utility_parameters,
        bpr=bpr_parameters,
        od=od_parameters,
        n_periods = len(np.unique(XT_train[:,:,-1].numpy().flatten()))
    )

    train_results_dfs['tvodlulpe'], test_results_dfs['tvodlulpe'] = tvodlulpe.train(
        XT_train, YT_train, XT_test, YT_test,
        optimizer=optimizer,
        # generalization_error={'train': False, 'validation': True},
        batch_size=_BATCH_SIZE,
        loss_weights= _LOSS_WEIGHTS,
        loss_metric=_LOSS_METRIC,
        momentum_equilibrium=_MOMENTUM_EQUILIBRIUM['tvodlulpe'],
        threshold_relative_gap=_RELATIVE_GAP,
        epochs=_EPOCHS)

    plot_predictive_performance(train_losses=train_results_dfs['tvodlulpe'], val_losses=test_results_dfs['tvodlulpe'],
                               xticks_spacing=_XTICKS_SPACING)

    plot_convergence_estimates(estimates=train_results_dfs['tvodlulpe'][['epoch', 'alpha', 'beta']],
                               xticks_spacing=_XTICKS_SPACING)

    sns.displot(pd.melt(pd.DataFrame({'alpha':tvodlulpe.alpha, 'beta': tvodlulpe.beta}), var_name = 'parameters'),
                x="value", hue="parameters", alpha = 0.8)

    top_q, total_trips_by_hour = plot_top_od_flows_periods(tvodlulpe,
                                                           period_keys = period_keys,
                                                           historic_od= fresno_network.q.flatten(),
                                                           period_feature='hour', top_k=20)

    # Compute utility parameters over time (heatmap) and value of travel time reliability (lineplot)
    theta_df = plot_utility_parameters_periods(tvodlulpe, period_keys = period_keys, period_feature='hour')

    plot_rr_by_period(tvodlulpe, period_keys, period_feature='hour')

    plot_convergence_estimates(estimates=train_results_dfs['tvodlulpe'].\
               assign(rr = train_results_dfs['tvodlulpe']['tt_sd']/train_results_dfs['tvodlulpe']['tt'])[['epoch','rr']],
                   xticks_spacing = _XTICKS_SPACING)
    plt.ylabel('average reliability ratio')

    sns.displot(pd.DataFrame({'fixed_effect':np.array(tvodlulpe.fixed_effect)}),
            x="fixed_effect", multiple="stack", kind="kde", alpha = 0.8)

    # print(f"theta = {dict(zip(utility_parameters.true_values.keys(), list(tvodlulpe.theta.numpy())))}")
    print(f"theta:\n\n\ {theta_df.assign(rr = theta_df['tt_sd']/theta_df['tt'])}")
    print(f"alpha = {np.mean(tvodlulpe.alpha): 0.2f}, beta  = {np.mean(tvodlulpe.beta): 0.2f}")
    print(f"Avg abs diff of observed and estimated OD: {np.mean(np.abs(tvodlulpe.q - fresno_network.q.flatten())): 0.2f}")
    print(f"Avg observed OD: {np.mean(np.abs(fresno_network.q.flatten())): 0.2f}")

## Write predictions

In [ ]:
predictions = pd.DataFrame({'link_key': list(fresno_network.links_keys) * Y_train.shape[0],
                            'observed_traveltime': Y_train[:, :, 0].numpy().flatten(),
                            'observed_flow': Y_train[:, :, 1].numpy().flatten()})

predictions['date'] = sorted(df[df.hour == 16].loc[df[df.hour == 16].year == 2019, 'date'])

# TODO: Write predictions for TVODLULPE model
for model in [lue,odlue,odlulpe]:

    predicted_flows = model.flows()
    predicted_traveltimes = model.traveltimes()

    predictions['predicted_traveltime_' + model.key] = np.tile(predicted_traveltimes, (Y_train.shape[0], 1)).flatten()
    predictions['predicted_flow_' + model.key] = np.tile(predicted_flows, (Y_train.shape[0], 1)).flatten()

predictions.to_csv(f"./output/tables/{datetime.now().strftime('%y%m%d%H%M%S')}_train_predictions_{'Fresno'}.csv")

## Write estimation results

In [ ]:
train_results_df, test_results_df \
    = map(lambda x: pd.concat([results.assign(model = model)[['model'] + list(results.columns)]
                               for model, results in x.items()],axis = 0), [train_results_dfs, test_results_dfs])

network_name = 'Fresno'

train_filename = f"{datetime.now().strftime('%y%m%d%H%M%S')}_train_results_{network_name}.csv"
test_filename = f"{datetime.now().strftime('%y%m%d%H%M%S')}_validation_results_{network_name}.csv"
train_results_df.to_csv(f"./output/tables/{train_filename}")
print(f'File {train_filename} was written')
test_results_df.to_csv(f"./output/tables/{test_filename}")
print(f'File {test_filename} was written')

## Summary of models parameters

In [ ]:
models = [lue,odlue,odlulpe, tvodlulpe]
results = pd.DataFrame({'parameter': [], 'model': []})

for model in models:
    results = results.append(pd.DataFrame(
        {'parameter': ['tt'] + features_Z +
                      ['rr'] +
                      ['fixed_effect_mean', 'fixed_effect_std',
                       'alpha_mean', 'alpha_std',
                       'beta_mean', 'beta_std',
                       'od_mean', 'od_std',],
         'values': list(np.mean(model.theta.numpy(),axis =0))  +
                   [float(model.get_parameters_estimates().eval('tt_sd/tt'))] +
                   [np.mean(model.fixed_effect),np.std(model.fixed_effect),
                    np.mean(model.alpha),np.std(model.alpha),
                    np.mean(model.beta),np.std(model.beta),
                    np.mean(model.q),np.std(model.q)]}).\
                             assign(model = model.key)
                             )

In [ ]:
results.pivot_table(index = ['parameter'], columns = 'model', values = 'values', sort=False).round(4)

## Summary of models goodness of fit

In [ ]:
models = [lue, odlue, odlulpe, tvodlulpe]
results_losses = pd.DataFrame({})
loss_columns = ['loss_flow', 'loss_tt', 'loss_eq_flow', 'loss_total']

for i, model in enumerate(models):

    results_losses_model = model.split_results(train_results_dfs[model.key])[1].assign(model = model.key)
    results_losses_model = results_losses_model[results_losses_model.epoch == results_losses_model.epoch.max()].iloc[[0]]
    results_losses = results_losses.append(results_losses_model)

results_losses[loss_columns] = (results_losses[loss_columns]-1)*100

In [ ]:
results_losses[['model'] + list(results_losses.columns)]

## Plot of convergence toward true rr across models

In [ ]:
train_estimates = {}
train_losses = {}

for model in models:
    train_estimates[model.key], train_losses[model.key] = model.split_results(results=train_results_dfs[model.key])

    train_estimates[model.key]['model'] = model.key

train_estimates_df = pd.concat(train_estimates.values())

train_estimates_df['rr'] = train_estimates_df['tt_sd']/train_estimates_df['tt']

estimates = train_estimates_df[['epoch','model','rr']].reset_index().drop('index',axis = 1)
estimates = estimates[estimates.epoch != 0]

fig, ax = plt.subplots(nrows=1, ncols=1)

g = sns.lineplot(data=estimates, x='epoch', hue='model', y='rr')

ax.set_ylabel('average reliability ratio')

plt.ylim(ymin=0)

plt.show()

## Plot of reliability ratio by hour for all models

In [ ]:
reliability_ratios = plot_rr_by_period_models(models, period_keys, period_feature='hour')
reliability_ratios.groupby('model')[['rr']].mean().round(4)

## Plot of total trips by hour for all models

In [ ]:
total_trips = plot_total_trips_models(models = models, period_feature = 'hour', period_keys = period_keys,
                                      historic_od = fresno_network.q.flatten())
total_trips.groupby('model')[['total_trips']].mean().round(1)